In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split

pre_trained = True

In [10]:

def distance_canberra_mat(d1, d2):
    ### Don't seems to work in scipy    
    divide = abs(d1) + abs(d2)    
    distance = np.sum(np.divide(abs(d1-d2), divide, where=divide!=0), axis=1)    
    return distance
    
def distance_cosine_mat(d1, d2):
    ### Don't seems to work in scipy
    val = np.sum(d1*d2, axis=1)
    divide = np.sqrt(np.sum((d1*d1), axis=1))*np.sqrt(np.sum((d2*d2), axis=1))
    return 1 - np.divide(val, divide)

def distance_khi2_mat_w(d1, d2, w):
    divide = (d1 + d2)*w    
    distance = np.sum(np.divide((d1-d2)**2, divide, where=divide!=0), axis=1)   
    return distance

def distance_khi2_mat(d1, d2):
    divide = d1 + d2    
    distance = np.sum(np.divide((d1-d2)**2, divide, where=divide!=0), axis=1)   
    return distance


def distance_braycurtis_mat(d1, d2):
    divide = np.sum(abs(d1 + d2), axis=1)
    distance = np.divide(np.sum(abs(d1-d2), axis=1), divide)   
    return distance


def distance_eucl_mat(d1, d2):
    distance = np.sqrt(np.sum((d1-d2)**2, axis=1))   
    return distance

In [ ]:
train_label = pd.read_csv("data/train_with_ids.csv")

In [ ]:
#train_label[["QS","QR"]].to_csv("data/train_labels.csv")

In [ ]:
train_data = None


if(pre_trained==False) :
    xtrain=pd.read_csv("data/xtrain_challenge.csv")
    train_data = xtrain
    
    cols_qs = list(xtrain.columns)[:13]
    cols_qr = list(xtrain.columns)[13:26]
    cols_S1_11 = list(xtrain.columns)[26:37]
    


    # On créé des features artificielles:
    # Q1-13 Q13-26 sont des identifiants uniques d'un visage
    # On suppose que Q1-13 et Q13-26 pourrait etre inversé
    # On regarde si on n'aurait pas des liaisons à faire en plus

    # Merge des colonnes

    train_data["QS"] = train_data[cols_qs].apply(
        lambda x: '_'.join(x.astype(str)),
        axis=1
    )

    # Merge des colonnes

    train_data["QR"] = train_data[cols_qr].apply(
        lambda x: '_'.join(x.astype(str)),
        axis=1
    )
    train_data['QS'] = train_data['QS'].apply(lambda x: hash(x))
    train_data['QR'] = train_data['QR'].apply(lambda x: hash(x))
    train_data[['QS','QR']].to_csv("data/train_labels.csv")
else:
    train_data = pd.read_csv("data/xtrain_challenge.csv")
    
  
    
cols_qs = list(train_data.columns)[:13]
cols_qr = list(train_data.columns)[13:26]
cols_S1_11 = list(train_data.columns)[26:37]
    

In [ ]:
train_data["canberra"]=distance_canberra_mat(train_data[cols_qr].values, train_data[cols_qs].values)
train_data["cosine"]=distance_cosine_mat(train_data[cols_qr].values, train_data[cols_qs].values)
train_data["khi2"]=distance_khi2_mat(train_data[cols_qr].values,train_data[cols_qs].values)
train_data["braycurtis"]=distance_braycurtis_mat(train_data[cols_qr].values,train_data[cols_qs].values)
train_data["euclidian"]=distance_eucl_mat(train_data[cols_qr].values,train_data[cols_qs].values)
ytrain=pd.read_csv("data/ytrain_challenge.csv")
xtrain = train_data

In [ ]:
train_data['QS'] = train_label['QS']
train_data['QR'] = train_label['QR']

In [ ]:
train_data.head(10)


In [ ]:
#train_data[train_data.label==1].groupby("QS")

In [ ]:
for c in list(xtrain.columns):    
    xtrain[c].plot(kind='hist', bins=30)
    plt.title(c)
    plt.show()


In [ ]:
train_data.describe()

In [ ]:
# Load test data
xtest = pd.read_csv('data/xtest_challenge.csv')

In [ ]:
xtest["canberra"]=distance_canberra_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["cosine"]=distance_cosine_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["khi2"]=distance_khi2_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["braycurtis"]=distance_braycurtis_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["euclidian"]=distance_eucl_mat(xtest[cols_qr].values,xtest[cols_qs].values)

In [ ]:
xtest.describe()

On va calculer des distances entre les features Q1-13 et Q13-26

https://www.cs.toronto.edu/~ranzato/publications/taigman_cvpr14.pdf


In [ ]:
# Calcul d'un SVM pour connaitre les coefficients des features

In [ ]:
from sklearn import svm
clf = svm.LinearSVC()

d1 = train_data[cols_qs].values
d2 = train_data[cols_qr].values

divide = d1 + d2 
X = np.divide((d1-d2)**2, divide, where=divide!=0)

X_SVM, X_SVM_test, y_SVM, y_SVM_test = train_test_split(X, ytrain, test_size=0.98) # SVM on 2% of data

X_SVM.shape

In [ ]:
clf.fit(X_SVM, y_SVM)

In [ ]:
w = clf.coef_

In [ ]:
xtest["khi2 W"]=distance_khi2_mat_w(xtest[cols_qr].values,xtest[cols_qs].values, w)
train_data["khi2_W"]=distance_khi2_mat_w(train_data[cols_qr].values,train_data[cols_qs].values,w)

In [ ]:
ytrain[xtrain.qr9 >xtrain.qs9.max()].index

In [ ]:
ytrain[xtrain.qr1 >xtrain.qs1.max()].index

In [ ]:
xtest[xtest.qr1>1].index

In [ ]:
xtest[xtest.qr9> xtrain.qs9.max()].index

In [ ]:
xtrain_clean = xtrain[xtrain.qr1<=1]



In [ ]:
xtrain_clean.describe()

In [ ]:
qs_unique = len(xtrain_clean.QS.unique())

##### for c in list(xtrain_clean.columns):
    xtrain_clean[c].plot(kind='hist', bins=30, logy=True)   
    plt.title(c)
    plt.show()    
    xtrain_clean[c].plot(kind='hist', bins=30)
    plt.title(c)
    plt.show()

In [ ]:
len(xtrain_clean.QR.unique())

In [ ]:
ytrain_clean = ytrain[xtrain.qr1 <=1]
yone = ytrain_clean[ytrain_clean==1]

import math

choosen = sk.utils.random.sample_without_replacement(qs_unique,qs_unique//8)

# get the hash of the data
qs = xtrain_clean.QS.iloc[choosen].values





In [ ]:
indices = np.in1d(xtrain_clean.QS.values, qs)

In [ ]:
xtrain_clean.QS[indices].count()

In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(xtrain_clean, ytrain[xtrain.qr1 <=1], test_size=0.33, random_state=42)

X_train= xtrain_clean[][~indices]
X_test=xtrain_clean[indices]
y_train= ytrain_clean[~indices]
y_test=ytrain_clean[indices]


In [ ]:
cols_active = list(train_data.columns)

remove_list=["QS","QR"] #, "qs1","qr1","qs2","qr2","qs3","qr3","qs12","qr12"]    
for i in remove_list: 
    try: 
        cols_active.remove(i) 
    except ValueError: 
        pass
    

In [ ]:

import lightgbm 
# making lgbm datasets for train and valid
d_train = lightgbm.Dataset(X_train[cols_active], y_train)
d_valid = lightgbm.Dataset(X_test[cols_active], y_test)
    
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_error',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 100,
    'max_bin':1024,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.2,
    'bagging_freq': 5,
    'learning_rate': 0.01,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       d_train,
                       valid_sets=d_valid,
                       num_boost_round=1000,
                       early_stopping_rounds=50)   
    # making prediciton for one column


In [ ]:

plt.rcParams['figure.figsize'] = [20, 10]
lightgbm.plot_importance(model)

In [ ]:
#model.save_model('lgbm_distance.txt')

soft = model.predict(X_test)

print('Fininshed Training')
soft

In [ ]:

yvalid = soft

In [ ]:
N = np.sum(y_test == 0).values
P = np.sum(y_test == 1).values



In [ ]:

yvalid_scoreordered = y_test.values[np.argsort(yvalid)]


FP = 0.0
TP = 0.0
val = 0
print(N)
print(P)
for i in range(len(y_test) - 1, -1, -1):        
    if (yvalid_scoreordered[i] == 1):
        TP = TP + 1
    else:
        FP = FP + 1
        #print(X_test[cols].iloc(i))
    if (FP / N > 10**-4):
        FP = FP - 1
        break
print("For the smallest FPR <= 10^-4 (i.e., ", FP / N, ") TPR = ", TP / P, ".", sep = "")

In [ ]:
# Load test data
xtest = pd.read_csv('data/xtest_challenge.csv')
xtest["canberra"]=distance_canberra_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["cosine"]=distance_cosine_mat(xtest[cols_qr].values, xtest[cols_qs].values)
xtest["khi2"]=distance_khi2_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["braycurtis"]=distance_braycurtis_mat(xtest[cols_qr].values,xtest[cols_qs].values)
xtest["euclidian"]=distance_eucl_mat(xtest[cols_qr].values,xtest[cols_qs].values)

In [ ]:
# Classify the provided test data
ytest = model.predict(xtest.values)
print(ytest.shape)
np.savetxt('ytest_lbgm_distance_s1.csv', ytest, fmt = '%1.15f', delimiter=',')